In [2]:
!uv pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Resolved 29 packages in 6.26s                                        
⠙ Preparing packages... (0/2)                                                   
⠙ Preparing packages... (0/2)----     0 B/784.03 MiB                    
⠙ Preparing packages... (0/2)---- 8.00 KiB/784.03 MiB                   
⠙ Preparing packages... (0/2)---- 16.00 KiB/784.03 MiB                  
⠙ Preparing packages... (0/2)---- 24.00 KiB/784.03 MiB                  
⠙ Preparing packages... (0/2)---- 32.00 KiB/784.03 MiB                  
⠙ Preparing packages... (0/2)---- 40.00 KiB/784.03 MiB                  
⠙ Preparing packages... (0/2)---- 48.00 KiB/784.03 MiB                  
⠙ Preparing packages... (0/2)---- 56.00 KiB/784.03 MiB                  
⠙ Preparing packages... (0/2)---- 64.00 KiB/784.03 MiB                  
⠙ Preparing packages... (0/2)---- 72.00 KiB/784.03 MiB                  
⠙ Preparing packages... (0/2)---- 80.00 KiB/784.03 MiB                  
⠙ Preparing packages... (0/2)---- 88.00 KiB/78

In [9]:
!uv add deep_translator
!git lfs install

Resolved 119 packages in 5ms
Uninstalled 3 packages in 59ms
Installed 3 packages in 131ms                               
 - fsspec==2025.5.1
 + fsspec==2025.3.0
 - torch==2.7.1
 + torch==1.13.1
 - torchvision==0.22.1
 + torchvision==0.14.1
Updated Git hooks.
Git LFS initialized.


In [10]:
import pandas as pd

df = pd.read_parquet('pubmed-sum-vi/train-00000-of-00005.parquet')

print(df.head())
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

                                             article  \
0  a recent systematic analysis showed that in 20...   
1  it occurs in more than 50% of patients and may...   
2  tardive dystonia ( td ) , a rarer side effect ...   
3  lepidoptera include agricultural pests that , ...   
4  syncope is caused by transient diffuse cerebra...   

                                            abstract  
0  background : the present study was carried out...  
1  backgroundanemia in patients with cancer who a...  
2  tardive dystonia ( td ) is a serious side effe...  
3  many lepidopteran insects are agricultural pes...  
4  we present an unusual case of recurrent cough ...  
Shape: (23985, 2)
Columns: ['article', 'abstract']


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the EnViT5 translation model
model_name = "models/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)  
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Translate English to Vietnamese
input_text = "en: I love programming."
inputs = tokenizer(input_text, return_tensors="pt", padding=True)
outputs = model.generate(inputs.input_ids, max_length=512)
translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(translated)

/home/nghiapd/Code/text-summerization-llm/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


vi: Tôi yêu lập trình.


In [1]:
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

/home/nghiapd/Code/text-summerization-llm/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_translation_model(model_name="models/envit5-translation", device="cuda" if torch.cuda.is_available() else "cpu"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    return tokenizer, model, device

In [3]:
def translate_texts(texts, tokenizer, model, device, prefix="en:"):
    results = []
    batch_size = 8  # Điều chỉnh batch_size tùy GPU/CPU
    for i in range(0, len(texts), batch_size):
        batch = [f"{prefix} {t}" for t in texts[i:i+batch_size]]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=512)
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        results.extend(decoded)
    return results

In [4]:
def translate_column(data, column_name, tokenizer, model, device, prefix):
    texts = data[column_name].astype(str).tolist()
    return translate_texts(texts, tokenizer, model, device, prefix=prefix)

In [5]:
def translate_and_save_parquet_with_model(file_path, output_dir='pubmed-sum-vi-translated', model_name="models/envit5-translation"):
    os.makedirs(output_dir, exist_ok=True)
    df = pd.read_parquet(file_path)
    print(f"Processing {file_path} with {len(df)} rows...")

    # Load model/tokenizer cho mỗi thread (an toàn hơn)
    def worker(column_name, prefix):
        tokenizer, model, device = load_translation_model(model_name)
        return translate_column(df, column_name, tokenizer, model, device, prefix)

    with ThreadPoolExecutor(max_workers=2) as executor:
        future_article = executor.submit(worker, 'article', 'en:')
        future_abstract = executor.submit(worker, 'abstract', 'en:')
        translated_articles = future_article.result()
        translated_abstracts = future_abstract.result()

    translated_df = pd.DataFrame({
        'article': translated_articles,
        'abstract': translated_abstracts
    })

    filename = os.path.basename(file_path)
    output_path = os.path.join(output_dir, f"translated_{filename}")
    translated_df.to_parquet(output_path, index=False)
    print(f"Saved translated file: {output_path}")
    return output_path


In [6]:
def process_all_files_with_model(input_dir='pubmed-sum-vi', model_name="models/envit5-translation"):
    parquet_files = [f for f in os.listdir(input_dir) if f.endswith('.parquet')]
    for file in sorted(parquet_files):
        file_path = os.path.join(input_dir, file)
        print(f"\nStarting translation of {file}...")
        translate_and_save_parquet_with_model(file_path, model_name=model_name)